<a href="https://colab.research.google.com/github/ManelSoengas/NLP_Curs/blob/main/Utilitzant_Transformers_Biblio_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tokenitzadors**

---
Si no hi ha un model d'idioma disponible en l'idioma que ens interessa, o si el nostre corpus és molt diferent del que s'ha entrenat el vostre model d'idioma, probablement necessitarem entrenar el model des de zero mitjançant un tokenitzador adaptat a les nostres dades. Això requerirà entrenar un nou tokenitzador pel conjunt de dades.


Es treballa amb un tokenizer personalitzat o amb un model, sovint es fa el següent:

1. Es carrega un dataset (pot ser de text, bibliografia, etc.).

1. Es construeix un corpus a partir d’aquest dataset.

1. Es fa servir aquest corpus per:

  * entrenar un tokenizer,

  * o tokenitzar el text per passar-lo a un model.

In [ ]:
!pip install datasets


from datasets import load_dataset

# This can take a few minutes to load, so grab a coffee or tea while you wait!
raw_datasets = load_dataset("code_search_net", "python")

In [ ]:
raw_datasets["train"]

In [ ]:
print(raw_datasets["train"][123456]["whole_func_string"])

# **Llavors… per què generar un corpus si ja tenim un dataset?**

---


El dataset pot contenir:

1. Metadades,

1. Camps diversos (autor, títol, any, resum…),

1. Informació textual estructurada o no estructurada.

Però el corpus generalment és:

Una llista de textos (normalment només cadenes de caràcters) que serveixen com a entrada per entrenar o aplicar un tokenizer.

In [5]:
training_corpus = (
    raw_datasets["train"][i : i + 1000]["whole_func_string"]
    for i in range(0, len(raw_datasets["train"]), 1000)
)

In [ ]:
gen = (i for i in range(10))
print(list(gen))
print(list(gen))

In [7]:
def get_training_corpus():
    return (
        raw_datasets["train"][i : i + 1000]["whole_func_string"]
        for i in range(0, len(raw_datasets["train"]), 1000)
    )


training_corpus = get_training_corpus()

# **Entrenant un nou tokenitzador**

---
Ara que tenim el nostre corpus en forma d'iterador de lots de textos, estem preparats per entrenar un nou tokenitzador. Per fer-ho, primer hem de carregar el tokenitzador que volem emparellar amb el nostre model (aquí, GPT-2)
